In [ ]:
!pip install PyGithub

# Project Plan for Setting Up a 3-Stage AWS CodePipeline

## Brainstorming

### Option 1: Manual Setup
* **Pros**: Full control over every step, no additional setup required.
* **Cons**: Time-consuming, prone to human error, not scalable.

### Option 2: Using AWS Management Console
* **Pros**: User-friendly interface, guided setup process.
* **Cons**: Still requires manual input, not as scalable as infrastructure as code.

### Option 3: Using Infrastructure as Code (IaC) Tools
* **Pros**: Highly scalable, can be version controlled, reduces human error.
* **Cons**: Requires knowledge of specific IaC language (e.g., AWS CloudFormation, Terraform).

## Evaluation

Given the need for a scalable and reliable solution, Option 3 (Using Infrastructure as Code Tools) seems to be the most suitable approach. It allows for version control and reduces the risk of human error.

## Expansion

If we choose to proceed with Option 3, we have further choices to make regarding which IaC tool to use. The most common options are AWS CloudFormation and Terraform. Both have their strengths and weaknesses, but given that we're working exclusively with AWS services, AWS CloudFormation might be the better choice.

## Decision

We will proceed with setting up the 3-stage AWS CodePipeline using AWS CloudFormation. This approach aligns with best practices for managing infrastructure and will provide the most efficient and reliable solution.

# Implementation

## AWS CloudFormation

We will use AWS CloudFormation to define the infrastructure needed for our 3-stage AWS CodePipeline. This will include the following resources:

1. **CodePipeline**: This will be the main resource that orchestrates the whole process. It will consist of three stages: Source, Build, and Deploy.
2. **CodeBuild**: This will be used in the Build stage of our pipeline. It will use a buildspec file to install Python requirements and build our application.
3. **Lambda**: This will be the resource that our pipeline deploys. It will be defined in a separate CloudFormation template which our pipeline will use to create or update the Lambda function.

## GitHub Python Module

We will use the GitHub Python module to interact with your GitHub repository. This will involve the following steps:

1. **Authentication**: We will authenticate with GitHub using a token.
2. **Repository Interaction**: We will interact with your repository to perform tasks such as creating a new file (for our CloudFormation templates) and updating existing files (like the README.md file).

Let's start by installing the GitHub Python module in our notebook.

# AWS CloudFormation Templates

We will develop the CloudFormation template resources separately and then aggregate them. This approach will help us manage our computational time more effectively.

## CodePipeline Template

This template will define the CodePipeline resource. It will consist of three stages: Source, Build, and Deploy. The Source stage will retrieve the source code from the GitHub repository. The Build stage will use CodeBuild to install Python requirements and build the application. The Deploy stage will use CloudFormation to create or update the Lambda function.

## CodeBuild Template

This template will define the CodeBuild resource. It will include the buildspec file that specifies the commands to run during the build stage. The buildspec file will use pip to install the Python requirements from the requirements.txt file.

## Lambda Template

This template will define the Lambda function that our pipeline deploys. It will be defined in a separate CloudFormation template which our pipeline will use to create or update the Lambda function.

## CodePipeline CloudFormation Template

Here is a high-level overview of what the CodePipeline CloudFormation template might look like:

```yaml
Resources:
  MyPipeline:
    Type: AWS::CodePipeline::Pipeline
    Properties:
      Stages:
        - Name: Source
          Actions:
            - Name: GitHub_Source
              ActionTypeId:
                Category: Source
                Owner: ThirdParty
                Provider: GitHub
                Version: 1
              OutputArtifacts:
                - Name: SourceOutput
              Configuration:
                Owner: <GitHub username>
                Repo: <GitHub repo>
                Branch: <GitHub branch>
                OAuthToken: !Ref GitHubToken
              RunOrder: 1
        - Name: Build
          Actions:
            - Name: CodeBuild_Build
              InputArtifacts:
                - Name: SourceOutput
              OutputArtifacts:
                - Name: BuildOutput
              ActionTypeId:
                Category: Build
                Owner: AWS
                Provider: CodeBuild
                Version: 1
              Configuration:
                ProjectName: !Ref CodeBuildProject
              RunOrder: 1
        - Name: Deploy
          Actions:
            - Name: CloudFormation_Deploy
              InputArtifacts:
                - Name: BuildOutput
              ActionTypeId:
                Category: Deploy
                Owner: AWS
                Provider: CloudFormation
                Version: 1
              Configuration:
                ActionMode: CREATE_UPDATE
                StackName: <CloudFormation stack name>
                TemplatePath: BuildOutput::template.yaml
              RunOrder: 1
```

This is a simplified version of the template and you will need to replace the placeholders (like `<GitHub username>`, `<GitHub repo>`, `<GitHub branch>`, and `<CloudFormation stack name>`) with your actual values. You will also need to define the `GitHubToken` and `CodeBuildProject` resources separately.

## CodePipeline CloudFormation Template with Parameters

Here is a high-level overview of what the CodePipeline CloudFormation template might look like with parameters instead of placeholders:

```yaml
Parameters:
  GitHubUsername:
    Type: String
  GitHubRepo:
    Type: String
  GitHubBranch:
    Type: String
  GitHubToken:
    Type: String
  CodeBuildProject:
    Type: String
  CloudFormationStackName:
    Type: String

Resources:
  MyPipeline:
    Type: AWS::CodePipeline::Pipeline
    Properties:
      Stages:
        - Name: Source
          Actions:
            - Name: GitHub_Source
              ActionTypeId:
                Category: Source
                Owner: ThirdParty
                Provider: GitHub
                Version: 1
              OutputArtifacts:
                - Name: SourceOutput
              Configuration:
                Owner: !Ref GitHubUsername
                Repo: !Ref GitHubRepo
                Branch: !Ref GitHubBranch
                OAuthToken: !Ref GitHubToken
              RunOrder: 1
        - Name: Build
          Actions:
            - Name: CodeBuild_Build
              InputArtifacts:
                - Name: SourceOutput
              OutputArtifacts:
                - Name: BuildOutput
              ActionTypeId:
                Category: Build
                Owner: AWS
                Provider: CodeBuild
                Version: 1
              Configuration:
                ProjectName: !Ref CodeBuildProject
              RunOrder: 1
        - Name: Deploy
          Actions:
            - Name: CloudFormation_Deploy
              InputArtifacts:
                - Name: BuildOutput
              ActionTypeId:
                Category: Deploy
                Owner: AWS
                Provider: CloudFormation
                Version: 1
              Configuration:
                ActionMode: CREATE_UPDATE
                StackName: !Ref CloudFormationStackName
                TemplatePath: BuildOutput::template.yaml
              RunOrder: 1
```

This version of the template uses parameters instead of placeholders. You can provide the actual values for these parameters when you create or update the CloudFormation stack.

## CodeBuild CloudFormation Template

Here is a high-level overview of what the CodeBuild CloudFormation template might look like:

```yaml
Parameters:
  ProjectName:
    Type: String
  BuildImage:
    Type: String
  BuildSpec:
    Type: String

Resources:
  MyCodeBuildProject:
    Type: AWS::CodeBuild::Project
    Properties:
      Name: !Ref ProjectName
      Source:
        Type: CODEPIPELINE
      Artifacts:
        Type: CODEPIPELINE
      Environment:
        Type: LINUX_CONTAINER
        ComputeType: BUILD_GENERAL1_SMALL
        Image: !Ref BuildImage
        EnvironmentVariables:
          - Name: AWS_ACCOUNT_ID
            Value: !Ref AWS::AccountId
          - Name: AWS_REGION
            Value: !Ref AWS::Region
      ServiceRole: !GetAtt CodeBuildServiceRole.Arn
      BuildSpec: !Ref BuildSpec

  CodeBuildServiceRole:
    Type: AWS::IAM::Role
    Properties:
      AssumeRolePolicyDocument:
        Version: '2012-10-17'
        Statement:
          - Effect: Allow
            Principal:
              Service: [codebuild.amazonaws.com]
            Action: ['sts:AssumeRole']
      Path: /
      Policies:
        - PolicyName: CodeBuildServiceRolePolicy
          PolicyDocument:
            Version: '2012-10-17'
            Statement:
              - Effect: Allow
                Action: ['*']
                Resource: ['*']
```

This version of the template uses parameters instead of placeholders. You can provide the actual values for these parameters when you create or update the CloudFormation stack. The `BuildSpec` parameter should contain the buildspec file as a string.

## BuildSpec for CodeBuild

The buildspec file specifies the commands that CodeBuild will run during the Build stage of our pipeline. In our case, we want it to install and run `cfn-lint` on the CloudFormation file to ensure it's valid.

Here is what the buildspec file might look like:

```yaml
version: 0.2

phases:
  install:
    runtime-versions:
      python: 3.8
    commands:
      - echo Installing cfn-lint...
      - pip install cfn-lint

  build:
    commands:
      - echo Running cfn-lint...
      - cfn-lint template.yaml

artifacts:
  files:
    - template.yaml
```

This buildspec file will install `cfn-lint` and then run it on `template.yaml`, which is assumed to be the name of our CloudFormation file. If the linting process finds any issues with the CloudFormation file, it will output the issues and the build will fail.

## Updated BuildSpec for CodeBuild

We can update the buildspec file to check if a `requirements.txt` file is present and, if it is, use `pip install -r` to install any Python requirements. Here is what the updated buildspec file might look like:

```yaml
version: 0.2

phases:
  install:
    runtime-versions:
      python: 3.8
    commands:
      - echo Installing cfn-lint...
      - pip install cfn-lint
      - if [ -f requirements.txt ]; then echo Installing Python requirements... && pip install -r requirements.txt; fi

  build:
    commands:
      - echo Running cfn-lint...
      - cfn-lint template.yaml

artifacts:
  files:
    - template.yaml
```

This updated buildspec file will check if a `requirements.txt` file is present and, if it is, install the Python requirements listed in the file. It will then run `cfn-lint` on `template.yaml` as before.

## Further Updated BuildSpec for CodeBuild

We can further update the buildspec file to only run `cfn-lint` on `template.yaml` if `template.yaml` is present. Here is what the further updated buildspec file might look like:

```yaml
version: 0.2

phases:
  install:
    runtime-versions:
      python: 3.8
    commands:
      - echo Installing cfn-lint...
      - pip install cfn-lint
      - if [ -f requirements.txt ]; then echo Installing Python requirements... && pip install -r requirements.txt; fi

  build:
    commands:
      - if [ -f template.yaml ]; then echo Running cfn-lint... && cfn-lint template.yaml; fi

artifacts:
  files:
    - template.yaml
```

This further updated buildspec file will only run `cfn-lint` on `template.yaml` if `template.yaml` is present.

## AWS Resource Naming Scheme

We will use a uniform naming scheme for AWS resources. This will make it easier to manage and identify resources. Here is the naming scheme we will use:

- CodeBuild project: `github-backup-build`
- CodePipeline: `github-backup-pipeline`
- AWS Lambda function: `github-backup-lambda-function`
- IAM role for the Lambda function: `github-backup-lambda-role`

We will use Boto3, AWS's Python SDK, to interact with these resources from our notebook.

## Updated CodeBuild CloudFormation Template

Here is the updated CodeBuild CloudFormation template using the new naming scheme:

```yaml
Parameters:
  BuildImage:
    Type: String
  BuildSpec:
    Type: String

Resources:
  GithubBackupBuild:
    Type: AWS::CodeBuild::Project
    Properties:
      Name: github-backup-build
      Source:
        Type: CODEPIPELINE
      Artifacts:
        Type: CODEPIPELINE
      Environment:
        Type: LINUX_CONTAINER
        ComputeType: BUILD_GENERAL1_SMALL
        Image: !Ref BuildImage
        EnvironmentVariables:
          - Name: AWS_ACCOUNT_ID
            Value: !Ref AWS::AccountId
          - Name: AWS_REGION
            Value: !Ref AWS::Region
      ServiceRole: !GetAtt GithubBackupBuildServiceRole.Arn
      BuildSpec: !Ref BuildSpec

  GithubBackupBuildServiceRole:
    Type: AWS::IAM::Role
    Properties:
      AssumeRolePolicyDocument:
        Version: '2012-10-17'
        Statement:
          - Effect: Allow
            Principal:
              Service: [codebuild.amazonaws.com]
            Action: ['sts:AssumeRole']
      Path: /
      Policies:
        - PolicyName: GithubBackupBuildServiceRolePolicy
          PolicyDocument:
            Version: '2012-10-17'
            Statement:
              - Effect: Allow
                Action: ['*']
                Resource: ['*']
```

This updated template uses the new naming scheme for the CodeBuild project and the associated IAM role.

## Lambda Function CloudFormation Template

Here is a high-level overview of what the Lambda function CloudFormation template might look like:

```yaml
Parameters:
  LambdaFunctionName:
    Type: String
  LambdaHandler:
    Type: String
  LambdaRuntime:
    Type: String
  LambdaRole:
    Type: String

Resources:
  GithubBackupLambdaFunction:
    Type: AWS::Lambda::Function
    Properties:
      FunctionName: !Ref LambdaFunctionName
      Handler: !Ref LambdaHandler
      Runtime: !Ref LambdaRuntime
      Role: !Ref LambdaRole
```

This version of the template uses parameters instead of placeholders. You can provide the actual values for these parameters when you create or update the CloudFormation stack.

## Lambda Function CloudFormation Template

Here is a high-level overview of what the Lambda function CloudFormation template might look like:

```yaml
Parameters:
  LambdaFunctionName:
    Type: String
  LambdaHandler:
    Type: String
  LambdaRuntime:
    Type: String
  LambdaRole:
    Type: String

Resources:
  GithubBackupLambdaFunction:
    Type: AWS::Lambda::Function
    Properties:
      FunctionName: !Ref LambdaFunctionName
      Handler: !Ref LambdaHandler
      Runtime: !Ref LambdaRuntime
      Role: !Ref LambdaRole
```

This version of the template uses parameters instead of placeholders. You can provide the actual values for these parameters when you create or update the CloudFormation stack.

## Thought Process for Project Implementation

### Brainstorming

- **Option 1:** Use AWS CloudFormation to create all the resources. This option would involve writing CloudFormation templates for each resource (CodeBuild, CodePipeline, Lambda function, etc.) and then using the AWS Management Console or AWS CLI to create a stack from these templates.
- **Option 2:** Use AWS SDKs (like Boto3 for Python) to create and manage the resources. This option would involve writing Python scripts to create, update, and delete resources as needed.
- **Option 3:** Use a combination of AWS CloudFormation and AWS SDKs. This option would involve using CloudFormation for resource creation and the SDKs for tasks that are more complex or not easily accomplished with CloudFormation alone.

### Evaluation

- **Option 1:** This option is straightforward and easy to manage, as all the resources are defined in one place (the CloudFormation templates). However, it might not be flexible enough for complex tasks, and it requires knowledge of the CloudFormation syntax and concepts.
- **Option 2:** This option is very flexible and can handle complex tasks, but it requires more programming knowledge and might be harder to manage, especially if the number of resources is large.
- **Option 3:** This option combines the advantages of the other two options. It uses CloudFormation for the easy management of resources and the SDKs for flexibility and complex tasks.

### Expansion

Based on the evaluation, Option 3 seems to be the best choice. We can use CloudFormation to define and create the resources, and then use the GitHub Python module and Boto3 to interact with the GitHub repository and the AWS resources, respectively.

### Decision-Making

We will proceed with Option 3. The next steps are to create the CloudFormation templates for the resources, use the GitHub Python module to update the README.md file in the GitHub repository, and use Boto3 to create and manage the AWS resources.

## Breaking Down the CodePipeline CloudFormation Template

Due to computational time limitations, we'll break down the CodePipeline CloudFormation template into smaller, more manageable parts. We'll create separate templates for the pipeline, stages, and actions, and then combine them programmatically.

### Pipeline Template

The pipeline template defines the basic structure of the pipeline, including its name and the IAM role it uses.

```yaml
Parameters:
  PipelineName:
    Type: String

Resources:
  GithubBackupPipeline:
    Type: AWS::CodePipeline::Pipeline
    Properties:
      Name: !Ref PipelineName
      RoleArn: !GetAtt GithubBackupPipelineRole.Arn
```

### Stages Template

The stages template defines the stages of the pipeline. Each stage has a name and a list of actions.

```yaml
Stages:
  - Name: Source
    Actions: []
  - Name: Build
    Actions: []
  - Name: Deploy
    Actions: []
```

### Actions Template

The actions template defines the actions that occur in each stage. Each action has a name, a type, and configuration details.

```yaml
Actions:
  - Name: GitHub_Source
    ActionTypeId:
      Category: Source
      Owner: ThirdParty
      Provider: GitHub
      Version: 1
    OutputArtifacts:
      - Name: SourceOutput
    Configuration:
      Owner: !Ref GitHubOwner
      Repo: !Ref GitHubRepo
      Branch: !Ref GitHubBranch
      OAuthToken: !Ref GitHubToken
```

We can create these templates separately and then combine them programmatically using Python.

In [ ]:
# Define the base templates

pipeline_template = {
  'Parameters': {
    'PipelineName': {
      'Type': 'String'
    }
  },
  'Resources': {
    'GithubBackupPipeline': {
      'Type': 'AWS::CodePipeline::Pipeline',
      'Properties': {
        'Name': {'Ref': 'PipelineName'},
        'RoleArn': {'Fn::GetAtt': ['GithubBackupPipelineRole', 'Arn']}
      }
    }
  }
}

stages_template = {
  'Stages': [
    {'Name': 'Source', 'Actions': []},
    {'Name': 'Build', 'Actions': []},
    {'Name': 'Deploy', 'Actions': []}
  ]
}

actions_template = {
  'Actions': [
    {
      'Name': 'GitHub_Source',
      'ActionTypeId': {
        'Category': 'Source',
        'Owner': 'ThirdParty',
        'Provider': 'GitHub',
        'Version': '1'
      },
      'OutputArtifacts': [
        {'Name': 'SourceOutput'}
      ],
      'Configuration': {
        'Owner': {'Ref': 'GitHubOwner'},
        'Repo': {'Ref': 'GitHubRepo'},
        'Branch': {'Ref': 'GitHubBranch'},
        'OAuthToken': {'Ref': 'GitHubToken'}
      }
    }
  ]
}

# Combine the templates

combined_template = {**pipeline_template, **stages_template, **actions_template}

combined_template

In [ ]:
# Define the actions for each stage

source_action = {
  'Name': 'GitHub_Source',
  'ActionTypeId': {
    'Category': 'Source',
    'Owner': 'ThirdParty',
    'Provider': 'GitHub',
    'Version': '1'
  },
  'OutputArtifacts': [
    {'Name': 'SourceOutput'}
  ],
  'Configuration': {
    'Owner': {'Ref': 'GitHubOwner'},
    'Repo': {'Ref': 'GitHubRepo'},
    'Branch': {'Ref': 'GitHubBranch'},
    'OAuthToken': {'Ref': 'GitHubToken'}
  }
}

build_action = {
  'Name': 'CodeBuild_Build',
  'ActionTypeId': {
    'Category': 'Build',
    'Owner': 'AWS',
    'Provider': 'CodeBuild',
    'Version': '1'
  },
  'InputArtifacts': [
    {'Name': 'SourceOutput'}
  ],
  'Configuration': {
    'ProjectName': {'Ref': 'CodeBuildProjectName'}
  }
}

deploy_action = {
  'Name': 'Lambda_Deploy',
  'ActionTypeId': {
    'Category': 'Invoke',
    'Owner': 'AWS',
    'Provider': 'Lambda',
    'Version': '1'
  },
  'Configuration': {
    'FunctionName': {'Ref': 'LambdaFunctionName'}
  }
}

# Add the actions to the stages

combined_template['Stages'][0]['Actions'].append(source_action)
combined_template['Stages'][1]['Actions'].append(build_action)
combined_template['Stages'][2]['Actions'].append(deploy_action)

combined_template

In [ ]:
import json
import yaml

# Convert the combined template to JSON
json_template = json.dumps(combined_template, indent=2)

# Convert the combined template to YAML
yaml_template = yaml.dump(combined_template, default_flow_style=False)

json_template, yaml_template

In [ ]:
import boto3

# Initialize the CloudFormation client
cf = boto3.client('cloudformation')

# Create a CloudFormation stack
response = cf.create_stack(
  StackName='GithubBackupPipeline',
  TemplateBody=json_template,
  Parameters=[
    {
      'ParameterKey': 'PipelineName',
      'ParameterValue': 'GithubBackupPipeline'
    }
  ],
  Capabilities=['CAPABILITY_NAMED_IAM']
)

response

In [ ]:
import os

# Get the AWS region from the environment variables
aws_region = os.environ['AWS_REGION']

# Initialize the CloudFormation client with the specified region
cf = boto3.client('cloudformation', region_name=aws_region)

# Create a CloudFormation stack
response = cf.create_stack(
  StackName='GithubBackupPipeline',
  TemplateBody=json_template,
  Parameters=[
    {
      'ParameterKey': 'PipelineName',
      'ParameterValue': 'GithubBackupPipeline'
    }
  ],
  Capabilities=['CAPABILITY_NAMED_IAM']
)

response

In [ ]:
# Correct the structure of the combined template

combined_template['Resources']['GithubBackupPipeline']['Properties']['Stages'] = combined_template.pop('Stages')

# Convert the corrected template to JSON
json_template = json.dumps(combined_template, indent=2)

# Create a CloudFormation stack with the corrected template
response = cf.create_stack(
  StackName='GithubBackupPipeline',
  TemplateBody=json_template,
  Parameters=[
    {
      'ParameterKey': 'PipelineName',
      'ParameterValue': 'GithubBackupPipeline'
    }
  ],
  Capabilities=['CAPABILITY_NAMED_IAM']
)

response

In [ ]:
# Remove the 'Actions' property from the top level of the template
combined_template.pop('Actions', None)

# Convert the corrected template to JSON
json_template = json.dumps(combined_template, indent=2)

# Create a CloudFormation stack with the corrected template
response = cf.create_stack(
  StackName='GithubBackupPipeline',
  TemplateBody=json_template,
  Parameters=[
    {
      'ParameterKey': 'PipelineName',
      'ParameterValue': 'GithubBackupPipeline'
    }
  ],
  Capabilities=['CAPABILITY_NAMED_IAM']
)

response

In [ ]:
# Define the 'GithubBackupPipelineRole' resource
combined_template['Resources']['GithubBackupPipelineRole'] = {
  'Type': 'AWS::IAM::Role',
  'Properties': {
    'AssumeRolePolicyDocument': {
      'Version': '2012-10-17',
      'Statement': [
        {
          'Effect': 'Allow',
          'Principal': {'Service': 'codepipeline.amazonaws.com'},
          'Action': 'sts:AssumeRole'
        }
      ]
    },
    'ManagedPolicyArns': [
      'arn:aws:iam::aws:policy/AmazonS3FullAccess',
      'arn:aws:iam::aws:policy/AWSCodeBuildAdminAccess',
      'arn:aws:iam::aws:policy/AWSCodePipelineFullAccess',
      'arn:aws:iam::aws:policy/AWSLambda_FullAccess'
    ]
  }
}

# Convert the corrected template to JSON
json_template = json.dumps(combined_template, indent=2)

# Create a CloudFormation stack with the corrected template
response = cf.create_stack(
  StackName='GithubBackupPipeline',
  TemplateBody=json_template,
  Parameters=[
    {
      'ParameterKey': 'PipelineName',
      'ParameterValue': 'GithubBackupPipeline'
    }
  ],
  Capabilities=['CAPABILITY_NAMED_IAM']
)

response

In [ ]:
# Initialize the SSM client
ssm = boto3.client('ssm', region_name=aws_region)

# Retrieve the GitHub token from the SSM parameter store
github_token = ssm.get_parameter(Name='/api/github', WithDecryption=True)['Parameter']['Value']

# Define the parameters for the CloudFormation stack
parameters = [
  {'ParameterKey': 'PipelineName', 'ParameterValue': 'GithubBackupPipeline'},
  {'ParameterKey': 'CodeBuildProjectName', 'ParameterValue': 'github-backup-build'},
  {'ParameterKey': 'GitHubOwner', 'ParameterValue': 'matthewhand'},
  {'ParameterKey': 'LambdaFunctionName', 'ParameterValue': 'github-backup-lambda-function'},
  {'ParameterKey': 'GitHubBranch', 'ParameterValue': 'main'},
  {'ParameterKey': 'GitHubToken', 'ParameterValue': github_token},
  {'ParameterKey': 'GitHubRepo', 'ParameterValue': 'github-backup'}
]

# Create a CloudFormation stack with the corrected template and the defined parameters
response = cf.create_stack(
  StackName='GithubBackupPipeline',
  TemplateBody=json_template,
  Parameters=parameters,
  Capabilities=['CAPABILITY_NAMED_IAM']
)

response

In [ ]:
# Define the parameters in the CloudFormation template
combined_template['Parameters'] = {
  'CodeBuildProjectName': {
    'Type': 'String',
    'Description': 'The name of the CodeBuild project'
  },
  'GitHubOwner': {
    'Type': 'String',
    'Description': 'The owner of the GitHub repository'
  },
  'LambdaFunctionName': {
    'Type': 'String',
    'Description': 'The name of the Lambda function'
  },
  'GitHubBranch': {
    'Type': 'String',
    'Description': 'The branch of the GitHub repository'
  },
  'GitHubToken': {
    'Type': 'String',
    'Description': 'The GitHub token for accessing the repository'
  },
  'GitHubRepo': {
    'Type': 'String',
    'Description': 'The name of the GitHub repository'
  }
}

# Convert the corrected template to JSON
json_template = json.dumps(combined_template, indent=2)

# Create a CloudFormation stack with the corrected template and the defined parameters
response = cf.create_stack(
  StackName='GithubBackupPipeline',
  TemplateBody=json_template,
  Parameters=parameters,
  Capabilities=['CAPABILITY_NAMED_IAM']
)

response

In [ ]:
# Add the 'PipelineName' parameter to the CloudFormation template
combined_template['Parameters']['PipelineName'] = {
  'Type': 'String',
  'Description': 'The name of the CodePipeline'
}

# Convert the corrected template to JSON
json_template = json.dumps(combined_template, indent=2)

# Create a CloudFormation stack with the corrected template and the defined parameters
response = cf.create_stack(
  StackName='GithubBackupPipeline',
  TemplateBody=json_template,
  Parameters=parameters,
  Capabilities=['CAPABILITY_NAMED_IAM']
)

response

In [ ]:
# Update the 'GitHubToken' parameter in the CloudFormation template
combined_template['Parameters']['GitHubToken'] = {
  'Type': 'AWS::SSM::Parameter::Value<String>',
  'Default': '/api/github',
  'Description': 'GitHub Token stored in SSM Parameter Store'
}

# Convert the corrected template to JSON
json_template = json.dumps(combined_template, indent=2)

# Create a CloudFormation stack with the corrected template and the defined parameters
response = cf.create_stack(
  StackName='GithubBackupPipeline',
  TemplateBody=json_template,
  Parameters=parameters,
  Capabilities=['CAPABILITY_NAMED_IAM']
)

response

In [ ]:
# Update the 'GithubBackupPipelineRole' resource to remove the AWSCodePipelineFullAccess policy
combined_template['Resources']['GithubBackupPipelineRole']['Properties']['ManagedPolicyArns'] = [
  'arn:aws:iam::aws:policy/AmazonS3FullAccess',
  'arn:aws:iam::aws:policy/AWSCodeBuildAdminAccess',
  'arn:aws:iam::aws:policy/AWSLambda_FullAccess'
]

# Convert the corrected template to JSON
json_template = json.dumps(combined_template, indent=2)

# Create a CloudFormation stack with the corrected template and the defined parameters
response = cf.create_stack(
  StackName='GithubBackupPipeline',
  TemplateBody=json_template,
  Parameters=parameters,
  Capabilities=['CAPABILITY_NAMED_IAM']
)

response